In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import time
import datetime
import ta
def upper_shadow(opener, close, high):
    if opener >= close:
        return abs(opener-high)
    else:
        return abs(close-high)

def lower_shadow(opener, close, low):
    if opener <= close:
        return abs(opener-low)
    else:
        return abs(close-low)

def real_body(opener, close):
    return abs(opener-close)

def total_dollar(av,bv,a,b):
    p = (a+b)/2.0
    v = av+bv
    return p*v

def log_dollar(a,b):
    p = (a+b)/2.0
    return np.log(p)

def volumen(av,bv):
    return av+bv

def price(a,b):
    g = (a+b)/2.0
    return b

#df = pd.read_csv('Dec.csv')
#df["Dollar"] = df.apply(lambda x: total_dollar(x.AskVolume,x.BidVolume,x.Ask,x.Bid), axis = 1)
def tick_preprocess(df):
    df["Price"] = df.apply(lambda x: price(x.Ask,x.Bid), axis = 1)
    return df
#df.to_csv("test_v6.csv",index=False)

In [2]:
def indicator_fill(df):
    a = ta.trend.ema_indicator(close=df.Close,window=5)
    f = ta.trend.sma_indicator(close=df.Close,window=50)
    i = ta.trend.sma_indicator(close=df.Close,window=20)
    c = ta.volatility.average_true_range(high=df.High,low=df.Low,close=df.Close)
    df["5_EMA"] = a
    df["50_SMA"] = f
    df["20_SMA"] = i
    df['ATR'] = c
    return df


In [3]:
import math
import statistics
def buy_quality(hi,lo,clo,start,bl):
    if bl =='buy' and abs(max(hi[1:]) - start) > abs(min(lo[1:]) - start):
        return "go buy"
    else:
        return "wrong go buy"

def sell_quality(hi,lo,clo,start,sl):
    if sl == 'sell' and abs(min(lo[1:]) - start) > abs(max(hi[1:]) - start):
        return "go sell"
    else:
        return "wrong go sell"

def mover(hi,lo,clo,start):
    up_move = max(hi[1:]) - start
    low_move = start - min(lo[1:])
    return up_move, low_move

def buy_calc_side(hi,lo,clo,start,atr):
    btgt = start + atr
    stgt = start - atr
    if max(hi[1:]) >= btgt:
        if min(lo[1:]) > stgt or np.argmin(np.array(lo[1:])) > np.argmax(np.array(hi[1:])):
            return 0
    if min(lo[1:]) <= stgt:
        if max(hi[1:]) < btgt or np.argmin(np.array(lo[1:])) < np.argmax(np.array(hi[1:])):
            return 1
    return 2

def meta_side(hi,lo,clo,start):
    if statistics.median(clo[1:]) < start:
        return 1
    else:
        return 0


def label_calc(df):
    buy_lab = []
    meta = []
    bq = []
    sq = []
    go = []
    profit = []
    loss = []
    k = 10
    for t in range(0,len(df)):
        if t >= len(df) - k:
            break
        a1 = df[t:t+k]
        a0 = df[t:t+k]
        st = a0.Close.tolist()[0]
        atr = a0.ATR.tolist()[0]
        hi = a1.High.tolist()
        lo = a1.Low.tolist()
        clo = a1.Close.tolist()
        buy_label = buy_calc_side(hi,lo,clo,st,atr)
        meta_ = meta_side(hi,lo,clo,st)
        meta += [meta_]
        buy_lab += [buy_label]
        p, l = mover(hi,lo,clo,st)
        profit += [p]
        loss += [l]



    if len(buy_lab) < len(df):
        buy_lab += ["Pass"] * (len(df)-len(buy_lab))
    if len(meta) < len(df):
        meta += ["Pass"] * (len(df)-len(meta))
    if len(profit) < len(df):
        profit += ["Pass"] * (len(df)-len(profit))
    if len(loss) < len(df):
        loss += ["Pass"] * (len(df)-len(loss))

    df["profit"] = profit
    df['loss'] = loss
    df["buy_label"] = buy_lab
    df['meta'] = meta

    return df

In [4]:
def df_vectorize(df):
    vec_load = []
    m5_vec = []
    buy_side = []
    sell_side = []
    b_quality = []
    s_quality = []
    take_profit = []
    stop_loss = []
    print(len(df))
    for h in range(10,len(df)):
        buff_df = df[h-10:h]
        if buff_df.buy_label.tolist()[-1] != "Skip":
            
            orig_close = buff_df.cl.tolist()
            orig_hi = buff_df.hi.tolist()
            orig_lo = buff_df.lo.tolist()
            orig_op = buff_df.op.tolist()
            close_log = buff_df.Close.tolist()
            hi = buff_df.High.tolist()
            lo = buff_df.Low.tolist()
            op = buff_df.Open.tolist()
            rb_vec = buff_df.rb.tolist()
            us_vec = buff_df.us.tolist()
            ls_vec = buff_df.ls.tolist()
            t_ma = buff_df['20_SMA'].tolist()
            f_ma = buff_df['5_EMA'].tolist()
            sma = buff_df.sma.tolist()
            fema = buff_df.fema.tolist()
            atr = buff_df.ATR.tolist()
            ma_diff = buff_df.ma_diff.tolist()
            tclose_diff = buff_df.tclose_diff.tolist()
            fclose_diff = buff_df.fclose_diff.tolist()
            med = statistics.median(buff_df.cl.tolist())
            maximum = max(buff_df.cl.tolist())
            minimum = min(buff_df.cl.tolist())
            stand_dev = statistics.stdev(buff_df.cl.tolist())
            
            buy_side += [buff_df.buy_label.tolist()[-1]]
            sell_side += [buff_df.sell_label.tolist()[-1]]
            b_quality += [buff_df.bu_quality.tolist()[-1]]
            s_quality += [buff_df.se_quality.tolist()[-1]]
            take_profit += [buff_df.Profit.tolist()[-1]]
            stop_loss += [buff_df.Loss.tolist()[-1]]

            #vec_load += [close_log + rb_vec + us_vec + ls_vec + ma_diff + tclose_diff + fclose_diff]
            m5_vec += [close_log + rb_vec + us_vec + ls_vec + ma_diff + tclose_diff + fclose_diff + t_ma + f_ma + sma + 
                     fema + atr + orig_close + orig_hi + orig_lo + orig_op + hi + lo + op]

        else:
            print('Skip')
    print('Vector_Done')
    vec_load = m5_vec
    return vec_load, buy_side, sell_side, b_quality, s_quality, take_profit, stop_loss
        
        

In [5]:
from fracdiff import FracdiffStat
cf = FracdiffStat()
hf = FracdiffStat()
of = FracdiffStat()
lf = FracdiffStat()
mf = FracdiffStat()


In [ ]:
import numpy as np
import pickle
from joblib import dump,load




buy_le = load('buy_le_SPX.joblib')
sell_le = load('sell_le_SPX.joblib')
bq_le = load('bq_le_SPX.joblib')
sq_le = load('sq_le_SPX.joblib')

In [31]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
df = pd.read_csv('5m_models\\SPX_Test.csv')
df = df[~((df.Open == df.Close) & (df.Open == df.High) & (df.Open == df.Low))]
df = indicator_fill(df)
df = label_calc(df)

print(len(df))
df = df[20:]
print(len(df))
df = df[df.buy_label!='Pass']
df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1).diff()
df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1).diff()
df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1).diff()
df['cl'] = df.Close
df['pct'] = scaler.fit_transform(np.nan_to_num(df.Close.pct_change().values).reshape(-1,1))
df['hi'] = scaler.fit_transform(np.nan_to_num(df.High.pct_change().values).reshape(-1,1))
df['lo'] = scaler.fit_transform(np.nan_to_num(df.Low.pct_change().values).reshape(-1,1))
df['op'] = scaler.fit_transform(np.nan_to_num(df.Open.pct_change().values).reshape(-1,1))
df['sma'] = scaler.fit_transform(np.nan_to_num(df['20_SMA'].pct_change().values).reshape(-1,1))
df['fema'] = scaler.fit_transform(np.nan_to_num(df['5_EMA'].pct_change().values).reshape(-1,1))


df['ma_diff'] = df['5_EMA'] - df['20_SMA']
df.ma_diff = scaler.fit_transform(np.nan_to_num(df.ma_diff.pct_change().values).reshape(-1,1))

df.us = scaler.fit_transform(np.nan_to_num(df.us.values.reshape(-1,1)))
df.ls = scaler.fit_transform(np.nan_to_num(df.ls.values.reshape(-1,1)))
df.rb = scaler.fit_transform(np.nan_to_num(df.rb.values.reshape(-1,1)))



#df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
df = df[1:]
print(len(df))


3405
3385
3374


In [32]:
df.to_csv('5m_models\\5m_spx_test.csv')

In [25]:
df = pd.read_csv('5m_models\\5m_spx_train.csv')
df['us1'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
df['ls1'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
df['rb1'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
df.to_csv('Check.csv')

In [ ]:

df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
v,bs,ss,bq,sq,tp,sl = df_vectorize(df)
df = df[10:]
v = np.array(v)




In [ ]:
v.shape

In [ ]:

v = np.nan_to_num(v)
v_v = v[:,40:50]
v_s = v[:,40:50]
v_bq = v[:,40:50]
v_sq = v[:,40:50]
vtp = v[:,40:50]
vsl = v[:,40:50]
bside_lab = buy_le.transform(bs)
sside_lab = sell_le.transform(ss)
bqside_lab = bq_le.transform(bq)
sqside_lab = sq_le.transform(sq)

In [ ]:
print(statistics.median(tp))
print(statistics.median(sl))
print(max(tp))
print(max(sl))

In [ ]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from tensorflow.keras.models import load_model
bmodel = load_model('5m_models\\best_bi-GRU_SPX_buy_model_5min.h5')
smodel = load_model('5m_models\\best_bi-GRU_SPX_sell_model_5min.h5')
bqmodel = load_model('5m_models\\best_bi-GRU_SPX_buy-quality_model_5min.h5')
sqmodel = load_model('5m_models\\best_bi-GRU_SPX_sell-quality_model_5min.h5')
tpmodel = load_model('5m_models\\best_bi-GRU_SPX_Price-Up-Move_model_5min.h5')
slmodel = load_model('5m_models\\best_bi-GRU_SPX_Price-Down-Move_model_5min.h5')
v_v = v_v.reshape(-1,v_v.shape[1],1)
v_s = v_s.reshape(-1,v_s.shape[1],1)
v_bq = v_bq.reshape(-1,v_bq.shape[1],1)
v_sq = v_sq.reshape(-1,v_sq.shape[1],1)
vtp = vtp.reshape(-1,vtp.shape[1],1)
vsl = vsl.reshape(-1,vsl.shape[1],1)

preds_out = bmodel.predict(v_v)
preds = np.argmax(preds_out, axis=1)
probs = np.max(preds_out, axis=1)

preds_out = smodel.predict(v_s)
spreds = np.argmax(preds_out, axis=1)
sprobs = np.max(preds_out, axis=1)

preds_out = bqmodel.predict(v_bq)
bqpreds = np.argmax(preds_out, axis=1)
bqprobs = np.max(preds_out, axis=1)

preds_out = sqmodel.predict(v_sq)
sqpreds = np.argmax(preds_out, axis=1)
sqprobs = np.max(preds_out, axis=1)

tppreds_out = tpmodel.predict(vtp)


slpreds_out = slmodel.predict(vsl)



print('Buy Side--------------------------')
print(accuracy_score(bside_lab, preds))
print(classification_report(bside_lab, preds))

print('Sell Side--------------------------')
print(accuracy_score(sside_lab, spreds))
print(classification_report(sside_lab, spreds))

print('Buy Quality Side--------------------------')
print(accuracy_score(bqside_lab, bqpreds))
print(classification_report(bqside_lab, bqpreds))

print('Sell Quality Side--------------------------')
print(accuracy_score(sqside_lab, sqpreds))
print(classification_report(sqside_lab, sqpreds))

In [ ]:
df['buy_side'] = buy_le.inverse_transform(preds)
df['buy_side_orig'] = buy_le.inverse_transform(bside_lab)
df['Buy_Probs'] = probs
df['sell_side'] = sell_le.inverse_transform(spreds)
df['sell_side_orig'] = sell_le.inverse_transform(sside_lab)
df['Sell_Probs'] = sprobs
df['buy_quality'] = bq_le.inverse_transform(bqpreds)
df['bq_Probs'] = bqprobs
df['sell_quality'] = sq_le.inverse_transform(sqpreds)
df['sq_Probs'] = sqprobs
df['Predicted_upmove'] = tppreds_out
df['Predicted_downmove'] = slpreds_out
df['Actual_upmove'] = tp
df['Actual_downmove'] = sl

df.to_csv('5m_models\\test_output_v2.csv', index=False)